In [49]:
import pandas as pd

In [50]:
df = pd.read_csv("spotify_millsongdata (1).csv")

In [52]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [53]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [54]:
df.shape

(57650, 4)

In [55]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [56]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [57]:
df.head(10)

,artist,song,text
0,Bob Seger,Turn On Your Love Light,"Without a warning you took my heart, \r\nYou ..."
1,Westlife,Never Knew I Was Losing You,Mark: \r\nI'm feeling so alone now tonight \...
2,Stevie Wonder,Jesus Children Of America,Hello Jesus \r\nJesus children \r\nJesus lov...
3,Barbra Streisand,I Got Rhythm,I got rhythm \r\nI got music \r\nI got my ma...
4,Bee Gees,Elisa,Elisa \r\nBee Gees \r\n \r\nAll that I need...
5,Leonard Cohen,Stories Of The Street,"The stories of the street are mine, the Spanis..."
6,Paul McCartney,Little Willow,"Bend, little willow \r\nWind's gonna blow you..."
7,W.A.S.P.,Hate To Love Me,You know I deceived ya \r\nYou see it's all i...
8,Tim Buckley,Strange Street Affair Under Blue,Just for you with your open hands \r\nWaiting...
9,Lady Gaga,Government Hooker,I can be good (If you just wanna be bad) \r\n...


In [58]:
df['text'][0]

"Without a warning you took my heart,  \r\nYou took it darling, tore it apart  \r\nYou left me sittin in the dark, cryin'  \r\nSaid your love for me was dyin'  \r\nI don't believe it, I think you outta turn it up  \r\nTurn on your love light  \r\nturn it on up  \r\njust a little bit  \r\n  \r\nI get a little lonely  \r\nIn the middle of the night  \r\nI need your lovin'  \r\nMake it all right  \r\nI need your love girl  \r\nOr else I ain't all right  \r\nI need your lovin'  \r\nOr else I ain't all right  \r\nBring it to me now  \r\nBring it to me now  \r\nBring it home darlin'  \r\nAn turn it on up now  \r\nJust a little bit  \r\nA little bit  \r\nTurn it up a bit\r\n\r\n"

In [59]:
# df = df.sample(5000)

In [60]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [61]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [62]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [63]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [65]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [66]:
similarity[0]

array([1.        , 0.0535053 , 0.00972802, ..., 0.07301012, 0.03651885,
       0.05166697])

In [67]:
df[df['song'] == 'Turn On Your Love Light']

,artist,song,text
0,Bob Seger,Turn On Your Love Light,"without a warn you took my heart , you took it..."


In [68]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [70]:
recommendation('Turn On Your Love Light')

["Little Bit O'soul",
 'Bit By Bit',
 'Give It Up',
 "Percy's Song",
 'Warning',
 'I Need Time',
 'Annie',
 'Little One',
 'You Need A Man-coming To You',
 'Fighter',
 'Down',
 'Just A Little Love',
 'Come Back Baby',
 'All I Want',
 "Keep Givin' Me Lovin'",
 "Put Your Lovin' On Me",
 'Nowhere Else To Turn',
 'Hard Day',
 'A Little Bit Of Soul',
 "Lovin' Money"]

In [ ]:
import pickle

pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))